# Import Modules

In [7]:
import pandas as pd
import re, time, requests, random
from datetime import datetime
# 爬蟲
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# import grequests # 看起要在.py才能用

# 非同步HTTP請求
from aiohttp import ClientSession, TCPConnector
# Python標準庫中提供的用於支援異步編程的模組, 可使用async和await關鍵字來定義異步協程
import asyncio
# 處理嵌套事件迴圈（nested event loop）的庫, 在Python中，通常只能有一個事件迴圈運行，
# 但某些情況下，比如在Jupyter Notebook中執行異步代碼時，可能會遇到嵌套事件迴圈的問題。
# nest_asyncio的作用就是解決這個問題，允許在已有事件迴圈的情況下再創建一個新的事件迴圈。
import nest_asyncio
# 它會修改當前執行環境，允許在已有事件迴圈的情況下再次建立一個新的事件迴圈，
# 通常用於處理一些特定的情況，例如在Jupyter Notebook中執行異步代碼。
nest_asyncio.apply()

# 執行

In [26]:
import crawler104_async, config.search_params
import importlib
importlib.reload(crawler104_async)
importlib.reload(config.search_params)

<module 'config.search_params' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\config\\search_params.py'>

In [27]:
from crawler104_async import JobScan104
from config.search_params import get_filter_params
import numpy as np

keywords_pattern = r'工程|資料|python|data|數據|後端'

# custom filter search
role = '全職'
keyword = "後端工程師 python"
cities = ['新北市', '台北市', '桃園市', '台中市']
isnew = '三日內'
jobexp = ['1年以下', '1-3年']
model = '列表'  # 一次能呈現比較多筆資料
order = '日期排序'
asc = '遞減'

filter_params = get_filter_params(role, keyword, cities, isnew, jobexp, model, order, asc)

# 碼表 Start
start_time = time.time()
# 建立物件
df = pd.DataFrame()
js104 = JobScan104(filter_params, keywords_pattern)
# 開始爬蟲
retry_count = 0
while True:
    try:
        raw_jobs = js104.search_job()
        break
    except:
        retry_count += 1
        if retry_count == 3:
            break
        print(f'執行錯誤, retry {retry_count}')

filter_jobs = js104.filter_job(raw_jobs)
# result_df = EJS.main(Job_list, DF)
# 碼表 End
print(f"花費 {np.round(time.time() - start_time)} 秒")

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&area=6001002000%2C6001001000%2C6001005000%2C6001008000&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 15/15 [00:30<00:00,  2.01s/page]


載入438筆資料 | 過濾後338筆 | 花費 42.61322331428528 秒


In [ ]:
async def scrape():
    print('test')


# raw_Job_list[20]
print(len(filter_jobs))
# 每個 batch 的 size (非同步每個 batch 處理)
batch_size = 30
# 確認 batches 數量
num_batches = (len(filter_jobs) + batch_size - 1) // batch_size

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(filter_jobs))
    # 每個 batch 頭尾 (0->29; 30->59; ......)
    jobs_batch = filter_jobs[start_idx:end_idx]
    # 取得異布事件循環
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(scrape())
    loop.close()

#     
#     loop = asyncio.get_event_loop()
#     results = loop.run_until_complete(self.scrape(Job_list_batch))
#     # loop.close()

#     for df in results:
#         if df is not None:
#             DF = pd.concat([DF, df], ignore_index=True)

# DF.to_csv(f"./output/JBLIST_{self.current_date}.csv", sep=',', index=False)
# return DF

In [2]:
class eJob_search104():
    
    
    def update_date(self, soup) -> str:
        update_date = soup.find("div", class_="job-header__title")
        return update_date.find('span').text.strip().replace('更新','') if update_date else '無'
    
    def company(self, soup) -> str:
        name = soup.find("div", class_="mt-3")
        return name.select_one('div > a').text.strip() if name else '無'
    
    def jd_info(self, soup) -> dict:
        result = {}
        JD = soup.find('div', class_='job-description-table row')
        if JD:
            jd_items = JD.find_all('div', recursive=False)
            if jd_items:
                    try:
                        job_content = jd_items[0].find('p').text if jd_items[0].find('p').text else '無'
                    except:
                        job_content = '無'

                    try:
                        job_category = ', '.join(i.text for i in jd_items[1].find_all('u')) if jd_items[1].find_all('u') else '無'
                    except:
                        job_category = '無'
                    
                    try:
                        salary = jd_items[2].find_all('div', recursive=False)[-1].text.strip() if jd_items[2].find_all('div', recursive=False)[-1].text.strip() else '無'
                    except:
                        salary = '無'
                    
                    try:
                        job_type = jd_items[3].find_all('div')[-1].text.strip() if jd_items[3].find_all('div')[-1].text.strip() else '無'
                    except:
                        job_type = '無'

                    try:
                        workingplace = jd_items[4].find_all('div')[-1].text.strip() if jd_items[4].find_all('div')[-1].text.strip() != '代企業徵才' else None
                    except:
                        workingplace = '無'

                    try:
                        management_responsibility = jd_items[6].find_all('div')[-1].text.strip() if jd_items[6].find_all('div')[-1].text.strip() else '無'
                    except:
                        management_responsibility = '無'
                    
                    try:
                        business_trip = jd_items[7].find_all('div')[-1].text.strip() if jd_items[7].find_all('div')[-1].text.strip() else '無'
                    except:
                        business_trip = '無'

                    try:
                        working_duration = jd_items[8].find_all('div')[-1].text.strip() if jd_items[8].find_all('div')[-1].text.strip() else '無'
                    except:
                        working_duration = '無'
                    
                    try:
                        Holiday_System = jd_items[9].find_all('div')[-1].text.strip() if jd_items[9].find_all('div')[-1].text.strip() else '無'
                    except:
                        Holiday_System = '無'
                    
                    try:
                        Working_date = jd_items[10].find_all('div')[-1].text.strip() if jd_items[10].find_all('div')[-1].text.strip() else '無'
                    except:
                        Working_date = '無'
                    
                    try:
                        ppl_required = jd_items[11].find_all('div')[-1].text.strip() if jd_items[11].find_all('div')[-1].text.strip() else '無'
                    except:
                        ppl_required = '無'


                    result = {
                                '工作內容': job_content,
                                '職務類別': job_category,
                                '工作待遇': salary,
                                '工作性質': job_type,
                                '上班地點': workingplace,
                                '管理責任': management_responsibility,
                                '出差外派': business_trip,
                                '上班時段': working_duration,
                                '休假制度': Holiday_System,
                                '可上班日': Working_date,
                                '需求人數': ppl_required
                            }
        return result
    
    def jr_info(self, soup) -> dict:
        result = {}
        JR = soup.find('div', class_= 'job-requirement-table row')
        JRO = soup.find('div', class_= 'job-requirement col opened')
        if JR:
            jr_items = JR.find_all('div', recursive=False)
            if jr_items:
                try:
                    work_exp = jr_items[0].find_all('div')[-1].text.strip() if jr_items[0].find_all('div')[-1].text.strip() else '無'
                except:
                    work_exp = '無'
                
                try:
                    academic_require = jr_items[1].find_all('div')[-1].text.strip() if jr_items[1].find_all('div')[-1].text.strip() else '無'
                except:
                    academic_require = '無'
                
                try:
                    department_require = jr_items[2].find_all('div')[-1].text.strip() if jr_items[2].find_all('div')[-1].text.strip() else '無'
                except:
                    department_require = '無'

                try:
                    language = jr_items[3].find('p').text.strip() if jr_items[3].find('p').text.strip() else '無'
                except:
                    language = '無'

                try:
                    tool = ', '.join(i.text for i in jr_items[4].find_all('u')) if jr_items[4].find_all('u') else '無'
                except:
                    tool = '無'
                
                try:
                    working_ability = jr_items[5].find_all('div')[-1].text.strip() if jr_items[5].find_all('div')[-1].text.strip() else '無'
                except:
                    working_ability = '無'
   
        try:
            others = JRO.find_all('div')[-1].text.strip() if JRO.find_all('div')[-1].text.strip() else '無'
        except:
            others = '無'

        if JR and JRO:
            result = {
                '工作經歷' : work_exp,
                '學歷要求' : academic_require,
                '科系要求' : department_require,
                '語文條件' : language,
                '擅長工具' : tool,
                '工作技能' : working_ability,
                '其他要求' : others
            }

        # result = {
        #     '工作經歷' : work_exp,
        #     '學歷要求' : academic_require,
        #     '科系要求' : department_require,
        #     '語文條件' : language,
        #     '擅長工具' : tool,
        #     '工作技能' : working_ability,
        #     '其他要求' : others
        # }
        return result
    
    async def fetch(self, session, url):
        async with session.get(url, headers = {'User-Agent':'GoogleBot'}) as response:
            return await response.text()

    async def get_job_info(self, item):
        try:
            title = item['title']
            Job_link = f"https:{item['href']}"
            connector = TCPConnector(limit=10)
            async with ClientSession(connector=connector) as session:
                html = await self.fetch(session, Job_link)
                soup = BeautifulSoup(html, 'lxml')

            Data = {
                '更新日期': [self.update_date(soup)],
                '職缺名稱': [title],
                '公司名稱': [self.company(soup)],
                '連結': [Job_link]
            }
            Data.update(self.jd_info(soup))
            Data.update(self.jr_info(soup))
            df = pd.DataFrame(Data, columns=['更新日期', '職缺名稱', '公司名稱', '工作內容', '職務類別', '工作待遇',
                                            '工作性質', '上班地點', '管理責任', '出差外派', '上班時段', '休假制度',
                                            '可上班日', '需求人數', '工作經歷', '學歷要求', '科系要求', '語文條件',
                                            '擅長工具', '工作技能', '其他要求', '連結'])
            return df
        except Exception as e:
            print("發生錯誤", e)
            return None

    async def scrape(self, Job_list):
        tasks = []
        semaphore = asyncio.Semaphore(10)  # Limit concurrent requests to 10

        for item in Job_list:
            async with semaphore:
                task = asyncio.ensure_future(self.get_job_info(item))
                tasks.append(task)

        return await asyncio.gather(*tasks)
    
    def main(self, Job_list: list, DF):
        print(len(Job_list))
        batch_size = 30
        num_batches = (len(Job_list) + batch_size - 1) // batch_size
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(Job_list))
            Job_list_batch = Job_list[start_idx:end_idx]
            loop = asyncio.get_event_loop()
            results = loop.run_until_complete(self.scrape(Job_list_batch))
            # loop.close()

            for df in results:
                if df is not None:
                    DF = pd.concat([DF, df], ignore_index=True)

        DF.to_csv(f"./output/JBLIST_{self.current_date}.csv", sep=',', index=False)
        return DF

In [8]:
import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)


# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。


Start asynchronous task 0
Start asynchronous task 1
Start asynchronous task 2
Start asynchronous task 3
Start asynchronous task 4
End asynchronous task 0
End asynchronous task 2
End asynchronous task 4
End asynchronous task 1
End asynchronous task 3
['Result of task 0', 'Result of task 1', 'Result of task 2', 'Result of task 3', 'Result of task 4']
